In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

credentials_location = '/root/.google/credentials/dezoomcamp-ikg-764b429f1606.json'

conf = SparkConf() \
    .setAppName('test') \
    .set("spark.jars", "/mnt/d/apache/spark-3.4.2-bin-hadoop3/jars/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
df_green = spark.read.parquet("gs://nytaxi-dezoomcamp/pq/green/*/*", header=True)
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')
df_green.columns

In [ ]:
df_yellow = spark.read.parquet("gs://nytaxi-dezoomcamp/pq/yellow/*/*", header=True)
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')
df_yellow.columns

In [ ]:
common_columns = [
    'VendorID',
    'pickup_datetime',
    'dropoff_datetime',
    'store_and_fwd_flag',
    'RatecodeID',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'payment_type',
    'congestion_surcharge'
]

In [ ]:
from pyspark.sql import functions as F

df_green_sel = \
    df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [ ]:
df_yellow_sel = \
    df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
# df_trips_data.registerTempTable('trips_data')
df_trips_data.createOrReplaceTempView ('trips_data')


In [ ]:
df_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 
    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,
    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
project_id = "dezoomcamp-ikg"
dataset_id = "trips_data_all"
table_id = "revenue_trips_data"

df_revenue.write \
    .format("bigquery") \
    .option("temporaryGcsBucket","dataproc-temp-asia-southeast2-79600360601-waczxn32") \
    .option("table", f"{project_id}.{dataset_id}.{table_id}") \
    .mode('Overwrite') \
    .save()


In [ ]:
spark.stop()